In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sympy as sy
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

sy.init_printing()

%matplotlib inline

In [ ]:
wakeup = pd.to_datetime("2025-05-01 7am")

pre_breakfast = pd.to_datetime("2025-05-01 8am")
post_breakfast = pd.to_datetime("2025-05-01 9am")

pre_lunch = pd.to_datetime("2025-05-01 12pm")
post_lunch = pd.to_datetime("2025-05-01 1am")

pre_dinner = pd.to_datetime("2025-05-01 6pm")
post_dinner = pd.to_datetime("2025-05-01 7pm")

bedtime = pd.to_datetime("2025-05-01 10pm")

## proof of concept: perturb and stablize system

### theory

In [ ]:
G_h, G, I, ISF, k_h, t = sy.symbols("G_h G I ISF k_h t")

In [ ]:
sy.Eq(sy.Derivative(G, t), k_h * t * (G_h - G))

### Implementation

In [ ]:
T = pd.date_range(start="2025-05-01", end="2025-05-02", freq="1min")
T

In [ ]:
T[0]

In [ ]:
stopper = 0

for i in T:
    print(i, type(i))
    print(i - T[0])
    if stopper > 5:
        break
    stopper += 1

In [ ]:
G_i = 60  # initial glucose
G_homeostatic_target = 100
k_glucose_return = 0.001  # "normal"; gets 60 -> 100 or 140 -> 100 over 60 minutes
t_step = 60  # 60s = 1 min

G_log = list()

dG_log = list()


for t_i in T:
    print(G_i)
    G_log.append(G_i)
    dG_log.append(k_glucose_return * t_step * (G_homeostatic_target - G_i))
    G_i += k_glucose_return * t_step * (G_homeostatic_target - G_i)

In [ ]:
pd.DataFrame({"glucose": G_log, "dG": dG_log}, index=T).plot()

So, it appears it comes back to center. Now for the insulin.

## now for insulin addition

In [ ]:
from collections import namedtuple

In [ ]:
InsulinType = namedtuple("InsulinType", ["insulin_name", "onset", "duration"])
InsulinDose = namedtuple("InsulinDose", ["InsulinType", "dose", "time"])

In [ ]:
glargine = InsulinType("glargine", pd.to_timedelta("30min"), pd.to_timedelta("20h"))
glargine

In [ ]:
glargine_dose_1 = InsulinDose(
    InsulinType=glargine, dose=10, time=pd.to_datetime("2025-05-01 8am")
)
glargine_dose_1

In [ ]:
glargine_dose_1.InsulinType.duration

Working nested datatype for adding insulin types and doses.

Let's add all insulins:

In [ ]:
degludec = InsulinType("degludec", pd.to_timedelta("10h"), pd.to_timedelta("42h"))
glargine_U_300 = InsulinType(
    "glargine_U_300", pd.to_timedelta("6h"), pd.to_timedelta("36h")
)
glargine_U_100 = InsulinType(
    "glargine_U_100", pd.to_timedelta("4h"), pd.to_timedelta("22h")
)
detemir = InsulinType("detemir", pd.to_timedelta("2h"), pd.to_timedelta("18h"))
regular_IV = InsulinType(
    "regular_IV", pd.to_timedelta("2min"), pd.to_timedelta("60min")
)
regular_SC = InsulinType("regular_SC", pd.to_timedelta("10min"), pd.to_timedelta("6h"))
RAI = InsulinType("RAI", pd.to_timedelta("10min"), pd.to_timedelta("4h"))
NPH = InsulinType("NPH", pd.to_timedelta("1h"), pd.to_timedelta("16h"))
[
    degludec,
    glargine_U_300,
    glargine_U_100,
    detemir,
    regular_IV,
    regular_SC,
    RAI,
    NPH,
]

In [ ]:
# basal bolus
basal_bolus = [
    InsulinDose(InsulinType=glargine_U_100, dose=20, time=wakeup),
    InsulinDose(InsulinType=RAI, dose=10, time=pre_breakfast),
    InsulinDose(InsulinType=RAI, dose=10, time=pre_lunch),
    InsulinDose(InsulinType=RAI, dose=10, time=pre_dinner),
]
basal_bolus

In [ ]:
I = np.zeros(len(T))